In [ ]:
# --- Install required packages ---
!pip install -q google-generativeai duckduckgo-search

In [ ]:
# --- Imports ---
import google.generativeai as genai
import json
import re
import textwrap
import time
from typing import Dict, List, TypedDict, Optional
from datetime import datetime
from duckduckgo_search import DDGS
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# --- Gemini API Setup ---
def setup_gemini_model():
    """
    Configure Gemini model for Google Colab.
    API key must be stored in Colab's secrets as 'GEMINI_API_KEY'.
    """
    try:
        from google.colab import userdata
        api_key = userdata.get('GEMINI_API_KEY')
        if not api_key:
            raise ValueError("GEMINI_API_KEY not found in Colab secrets.")
    except Exception:
        # Fallback: Prompt user for API key
        api_key = input("Please enter your Gemini API key: ").strip()
        if not api_key:
            raise ValueError("No Gemini API key provided.")

    genai.configure(api_key=api_key)

    generation_config = {
        "temperature": 0.7,
        "top_p": 1,
        "top_k": 1,
        "max_output_tokens": 2048,
    }
    safety_settings = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    ]
    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        generation_config=generation_config,
        safety_settings=safety_settings
    )
    return model

In [ ]:
# --- Initialize Model ---
model = setup_gemini_model()

In [ ]:
# --- Agent State ---
class AgentState(TypedDict):
    question: str
    initial_response: str
    critiques: List[str]
    refined_responses: List[str]
    search_results: List[Dict]
    current_response: str
    iteration: int
    is_satisfactory: bool
    max_iterations: int
    final_response: Optional[str]
    completed_at: Optional[str]
    total_iterations: Optional[int]

In [ ]:
# --- Web Search Tool ---
def web_search(query: str, max_results: int = 5) -> List[Dict]:
    """Perform web search using DuckDuckGo."""
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=max_results))
            return results
    except Exception as e:
        print(f"[Web Search Error]: {e}")
        return []

In [ ]:
# --- Gemini Response Generation ---
def generate_response(prompt: str, model=model) -> str:
    """Generate response using Gemini model."""
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"[Gemini Error]: {str(e)}")
        return f"Error generating response: {str(e)}"

In [ ]:
# --- Self-Critique ---
def self_critique(question: str, response: str, search_results: Optional[List[Dict]] = None) -> str:
    """Critique the model's response."""
    critique_prompt = f"""
You are a critical reviewer. Analyze the response to the question and provide constructive criticism.

QUESTION: {question}

RESPONSE: {response}

{"ADDITIONAL CONTEXT FROM SEARCH: " + json.dumps(search_results, ensure_ascii=False) if search_results else ""}

Please critique:
1. Factual accuracy and errors
2. Completeness
3. Clarity and structure
4. Missing information
5. Suggestions for improvement

CRITIQUE:
"""
    return generate_response(critique_prompt)

In [ ]:
# --- Response Refinement ---
def refine_response(question: str, current_response: str, critique: str, search_results: Optional[List[Dict]] = None) -> str:
    """Refine response based on critique and web search."""
    refinement_prompt = f"""
Refine your response to the question using the provided critique and search results.

QUESTION: {question}
CURRENT RESPONSE: {current_response}
CRITIQUE: {critique}
{"SEARCH RESULTS FOR VALIDATION: " + json.dumps(search_results, ensure_ascii=False) if search_results else ""}

Instructions:
- Address all critique points
- Incorporate relevant facts from search results
- Maintain clarity, professionalism, accuracy, and completeness

REFINED RESPONSE:
"""
    return generate_response(refinement_prompt)

In [ ]:
# --- Satisfaction Evaluation ---
def evaluate_satisfaction(question: str, response: str, critique_history: List[str]) -> bool:
    """Evaluate if the response is satisfactory."""
    evaluation_prompt = f"""
Evaluate if the following response satisfactorily answers the question.

QUESTION: {question}
RESPONSE: {response}
CRITIQUE HISTORY: {json.dumps(critique_history, ensure_ascii=False)}

Assess:
1. Does it fully answer the question?
2. Is it factually accurate?
3. Is it well-structured and clear?
4. Are previous critique points resolved?

Answer with ONLY "YES" or "NO" and a brief explanation.

EVALUATION:
"""
    evaluation = generate_response(evaluation_prompt)
    return "YES" in evaluation.upper() and ("thorough" in evaluation.lower() or "complete" in evaluation.lower())

In [ ]:
# --- Search Query Generation ---
def generate_search_queries(question: str, response: str, critique: str) -> List[str]:
    """Generate search queries for fact-checking."""
    search_prompt = f"""
Based on the question, response, and critique, generate 3 search queries to fact-check and improve the response.

QUESTION: {question}
RESPONSE: {response}
CRITIQUE: {critique}

Return ONLY the queries as a JSON list: ["query1", "query2", "query3"]

SEARCH QUERIES:
"""
    queries_text = generate_response(search_prompt)
    try:
        queries = json.loads(queries_text)
    except Exception:
        queries = re.findall(r'"([^"]*)"', queries_text)
        if not queries:
            queries = [f"fact check: {question}", f"latest research: {question}", question]
    return queries[:3]

In [ ]:
# --- Reflexion Workflow ---
def reflexion_workflow(question: str, max_iterations: int = 3) -> Dict:
    """Run the Reflexion loop to answer a question."""
    state = AgentState(
        question=question,
        initial_response="",
        critiques=[],
        refined_responses=[],
        search_results=[],
        current_response="",
        iteration=0,
        is_satisfactory=False,
        max_iterations=max_iterations,
        final_response=None,
        completed_at=None,
        total_iterations=None
    )

    print(f"\n🧠 Question: {question}")
    print("=" * 60)
    # Step 1: Initial Response
    print("🔄 Generating initial response...")
    state['current_response'] = generate_response(f"Please provide a comprehensive answer to the following question: {question}")
    state['initial_response'] = state['current_response']
    state['refined_responses'].append(state['current_response'])

    # Reflexion Loop
    while state['iteration'] < state['max_iterations'] and not state['is_satisfactory']:
        state['iteration'] += 1
        print(f"\n🔄 Iteration {state['iteration']} / {state['max_iterations']}")

        # Step 2: Critique
        print("🔍 Critiquing response...")
        critique = self_critique(state['question'], state['current_response'], state['search_results'] if state['search_results'] else None)
        state['critiques'].append(critique)

        # Step 3: Search & Validation
        if state['iteration'] > 1 or "fact" in critique.lower() or "accurate" in critique.lower():
            print("🌐 Generating search queries...")
            queries = generate_search_queries(state['question'], state['current_response'], critique)
            print(f"Search queries: {queries}")
            new_search_results = []
            for query in queries:
                results = web_search(query, max_results=2)
                new_search_results.extend(results)
                time.sleep(1)  # Polite delay
            state['search_results'].extend(new_search_results)
            print(f"🔎 {len(new_search_results)} new search results added.")

        # Step 4: Refine Response
        print("🛠️ Refining response...")
        refined_response = refine_response(
            state['question'],
            state['current_response'],
            critique,
            state['search_results'] if state['search_results'] else None
        )
        state['current_response'] = refined_response
        state['refined_responses'].append(refined_response)

        # Step 5: Evaluate Satisfaction
        print("✅ Evaluating response quality...")
        state['is_satisfactory'] = evaluate_satisfaction(
            state['question'],
            state['current_response'],
            state['critiques']
        )
        print("🎉 Satisfactory response!" if state['is_satisfactory'] else "⚠️ Needs improvement.")

    # Finalize State
    state['final_response'] = state['current_response']
    state['completed_at'] = datetime.now().isoformat()
    state['total_iterations'] = state['iteration']

    return state

In [ ]:
# --- Display Results ---
def format_response(response: Dict) -> str:
    """Formatted output for display."""
    output = [
        f"QUESTION: {response['question']}",
        "=" * 60,
        f"FINAL RESPONSE (after {response['total_iterations']} iterations):",
        "-" * 40,
        textwrap.fill(response['final_response'], width=80),
        "\nCRITIQUES:",
        "-" * 40
    ]
    for i, critique in enumerate(response['critiques'], 1):
        output.append(f"Critique {i}: {textwrap.shorten(critique, width=100, placeholder='...')}")
    return "\n".join(output)

def save_results(results: Dict, filename: str = None):
    """Save results to a JSON file."""
    if not filename:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"reflexion_agent_results_{timestamp}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"Results saved to {filename}")

In [ ]:
# --- Demonstration ---
def demonstrate_workflow():
    """Run demo with example questions."""
    example_questions = [
        "What are the latest advancements in quantum computing for 2024?",
        "Explain the implications of CRISPR gene editing technology on modern medicine.",
        "How is artificial intelligence being used in climate change research?"
    ]
    print("\n🤖 Reflexion AI Agent Demo")
    print("=" * 60)
    for i, question in enumerate(example_questions, 1):
        print(f"\nExample {i}: {question}")
        print("-" * 40)
        result = reflexion_workflow(question, max_iterations=2)
        print(f"\n✅ Final response after {result['total_iterations']} iterations:\n{'='*50}")
        print(textwrap.fill(result['final_response'], width=80))
        initial_len = len(result['initial_response'])
        final_len = len(result['final_response'])
        print(f"\n📊 Improvement: {initial_len} ➔ {final_len} characters ({final_len/initial_len:.1f}x detail)")
        if result['search_results']:
            print(f"🔍 Used {len(result['search_results'])} search results for validation")
        print("\n" + "=" * 60)

In [ ]:
# --- Interactive Mode ---
def interactive_mode():
    """Interactive Q&A mode."""
    print("\n🤖 Reflexion AI Agent - Interactive Mode")
    print("Type 'quit' to exit")
    print("=" * 50)
    while True:
        question = input("\n📝 Enter your question: ").strip()
        if question.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
        if not question:
            continue
        print("Processing your question...")
        start_time = time.time()
        try:
            result = reflexion_workflow(question, max_iterations=2)
            end_time = time.time()
            print(f"\n✅ Completed in {end_time - start_time:.1f} seconds")
            print(format_response(result))
            save = input("\n💾 Save these results? (y/n): ").lower()
            if save.startswith('y'):
                save_results(result)
        except Exception as e:
            print(f"❌ Error: {str(e)}")

In [ ]:
# --- Uncomment below to run demo or interactive mode ---
demonstrate_workflow()
#interactive_mode()